In [1]:
# home
import os
import ipywidgets as widgets
#SR --start
from ipywidgets import Button, GridBox, Layout, ButtonStyle
#SR --end
import ipyfilechooser
from IPython.display import display, clear_output, Markdown
import platform
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')

workingDirectory_home = os.path.expanduser("~")

In [2]:
image_home = open("logo.png", "rb").read()
img_home = widgets.Image(
    value=image_home,
    format='png',
    width=900,
    height=800,
)

In [3]:
def proceedFunction_home(change):
    global workingDirectory_home
    
    workingDirectory_home = workingDirectoryButton_home.selected_path
    os.chdir(workingDirectory_home)
    
    matrixButton_raw.default_path = workingDirectory_home
    matrixButton_ensemble.default_path = workingDirectory_home
    jdButton_ensemble.default_path = workingDirectory_home
    fileDirectoryButton_dtrend.default_path = workingDirectory_home
    uploadButton_periodogram.default_path = workingDirectory_home
    fileDirectoryButton_feature.default_path = workingDirectory_home
    fileNameButton_prediction.default_path = workingDirectory_home
    customModelButton_prediction.default_path = workingDirectory_home
    
    proceedButton_raw.disabled = False
    processDataButton_ensemble.disabled = False
    uploadProceedButton_dtrend.disabled = False
    uploadProceedButton_periodogram.disabled = False
    uploadProceedButton_feature.disabled = False
    proceedButton_prediction.disabled = False
    
    with proceedOut_home:
        clear_output()
        print(colored(f"Selected Directory: {workingDirectory_home}", 'cyan', attrs=['bold']))

In [4]:
def shutdownFunction_home(change):
    with shutdownOut_home:
        clear_output()
        print(f"Logged Out! Close the window!")
        
    if platform.system() == "Windows":
        os.system("taskkill /IM python.exe")
    else:
        os.system("kill $(pgrep voila)")

In [5]:
workingDirectoryButton_home = ipyfilechooser.FileChooser(os.path.expanduser("~"))
workingDirectoryButton_home.title = '<b>Setup Working Directory</b>'
workingDirectoryButton_home.show_only_dirs = True

proceedButton_home = widgets.Button(description="Setup", icon='home', button_style='info')
proceedOut_home = widgets.Output()

shutdownButton_home = widgets.Button(description="Shut Down", button_style='danger', tooltip='Close ASIVA', icon='power-off')
shutdownOut_home = widgets.Output()
download_home = widgets.Output()

with proceedOut_home:
    clear_output()
    print(colored(f"Selected Directory: {workingDirectory_home}", 'cyan', attrs=['bold']))

with download_home:
    clear_output()
    display(Markdown("[Download Example Data](https://chat-forms.com/forms/1625257028718-ltv/?form)"))

In [6]:
proceedButton_home.on_click(proceedFunction_home)
shutdownButton_home.on_click(shutdownFunction_home)

In [7]:
#homeTab = widgets.VBox([workingDirectoryButton_home, proceedButton_home, proceedOut_home, img_home, widgets.HBox([download_home, shutdownButton_home]), shutdownOut_home])

#SR --start
left = widgets.VBox([workingDirectoryButton_home, proceedButton_home, proceedOut_home])
right_top = widgets.Box(children=[shutdownButton_home,shutdownOut_home],layout=Layout(display='flex',flex_flow='column',align_items='flex-end'))
right = widgets.Box(children=[right_top,download_home],layout=Layout(display='flex',flex_flow='column',align_items='flex-end',justify_content='space-between'))
homeTab = widgets.GridBox(children=[left,img_home,right],layout=Layout(grid_template_columns='30% 50% 20%'))
#SR --end

In [8]:
# Data Processing Tab
import os
import sys
import ipywidgets as widgets
import ipyfilechooser
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import bqplot.pyplot as plt
import bqplot as bq
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')
import time, datetime

In [9]:
def fileInput_raw():
    
    if "01Data_Processing" not in os.listdir(workingDirectory_home):
        os.mkdir("01Data_Processing")
        
    if "plots" not in os.listdir("./01Data_Processing/"):
        os.mkdir("01Data_Processing/plots")
        
    if "logs" not in os.listdir("./01Data_Processing/"):
        os.mkdir("01Data_Processing/logs")
    
    if "output" not in os.listdir("./01Data_Processing/"):
        os.mkdir("01Data_Processing/output")
    
    global df_raw
    global jd
#     global directory_raw
#     directory_raw = workingDirectoryButton_raw.selected_path
#     os.chdir(directory_raw)
    df_raw = pd.read_csv(matrixButton_raw.selected_path + "/" + matrixButton_raw.selected_filename, header=None)
    df_raw.columns = range(1, df_raw.shape[1] + 1)
    df_raw = df_raw.replace([np.inf, -np.inf, "nan "], np.nan)
    
    global removedColumnList
    removedColumnList = []
    global removedRowList
    removedRowList = []

In [10]:
def dataInformation(change):
    global df_raw
    fileInput_raw()
    with dataInformationOut_raw:
        clear_output()
        print(colored(f"Obs, Stars: {df_raw.shape}", 'green', attrs=['bold']))
        print(colored(f"Missing Obs: {df_raw.isnull().sum().sum()}", 'green', attrs=['bold']))
    
    filterbox_column_raw.disabled = False
    checkMissingButton_column_raw.disabled = False

    removeColumnBox_raw.disabled = False 
    removeButton_column_raw.disabled = False 

    filterbox_row_raw.disabled = False 
    checkMissingButton_row_raw.disabled = False
  
    removeRowBox_raw.disabled = False 
    removeButton_row_raw.disabled = False 
    
    imputeDropdown_raw.disabled = False
    imputeButton_raw.disabled = False
    constantImputeField_raw.disabled = False
    
    proceedButton2_raw.disabled = False
    
    if df_raw.isnull().sum().sum() == 0:
        imputeDropdown_raw.disabled = True
        imputeButton_raw.disabled = True
        constantImputeField_raw.disabled = True
#     outlierButton_raw.disabled = False

In [11]:
def checkMissingFunction_column(change):
    global df_raw
    
    missIndex = []
    missCount = []

    x = df_raw.isnull().sum()

    #for i in x:
        #if i > float(filterbox_column_raw.value)/100 * df_raw.shape[0]:
            #missIndex.append(x.index[x==i][0])
            #missCount.append(i)
            
#SR --start
    c = 0
    for i in x:
        c = c+1
        if i > float(filterbox_column_raw.value)/100 * df_raw.shape[0]:
            if i != 0:
                missIndex.append(c)
                missCount.append(i)
#SR --end

    missingData = pd.DataFrame({"Star ID": missIndex, "Missing Obs": missCount})
    missingData.drop_duplicates(keep=False, inplace=True)
    with missingValueOut_column_raw:
        clear_output()
        print(missingData.to_string(index=False))

In [12]:
def removeMissingFunction_column(change):
    global df_raw
    global removedColumnList
    try:
        global df_raw
        x = removeColumnBox_raw.value
        x = x.split(",")
        column = [int(i) for i in x]
        df_raw = df_raw.drop(column, axis=1)
        for i in column:
            removedColumnList.append(i)
        with removeColumnOut_raw:
            clear_output()
            print(colored(f"Star ID {column} removed", 'green', attrs=['bold']))
    except Exception as e:
        with removeColumnOut_raw:
            clear_output()
            print(colored(f"Not removed", 'red', attrs=['bold']))

In [13]:
def checkMissingFunction_row(change):
    global df_raw
    
    missIndex = []
    missCount = []

    x = df_raw.isnull().sum(axis=1)

    #for i in x:
        #if i > float(filterbox_row_raw.value)/100 * df_raw.shape[1]:
            #missIndex.append(x.index[x==i][0])
            #missCount.append(i)
#SR --start
    r = 0
    for i in x:
        r = r+1
        if i > float(filterbox_row_raw.value)/100 * df_raw.shape[1]:
            if i != 0:
                missIndex.append(r)
                missCount.append(i)
#SR --end

    missingData = pd.DataFrame({"Night": missIndex, "Missing Obs": missCount})
    missingData.drop_duplicates(keep=False, inplace=True)
    with missingValueOut_row_raw:
        clear_output()
        print(missingData.to_string(index=False))

In [14]:
def removeMissingFunction_row(change):
    global removedRowList
    
    try:
        global df_raw
        x = removeRowBox_raw.value
        x = x.split(",")
        row = [int(i) for i in x]
        df_raw = df_raw.drop(row, axis=0)
        for i in row:
            removedRowList.append(i)
        with removeRowOut_raw:
            clear_output()
            print(colored(f"Obs {row} removed", 'green', attrs=['bold']))
    except Exception as e:
        with removeRowOut_raw:
            clear_output()
            print(colored(f"Not removed", 'red', attrs=['bold']))

In [15]:
def imputeFunction(change):
    global df_raw
    columns = df_raw.columns
    if imputeDropdown_raw.value == "Mean":
        
        imputer = SimpleImputer(strategy="mean")
        imputer.fit(df_raw)
        df_raw = imputer.transform(df_raw)
        df_raw = pd.DataFrame(df_raw, columns=columns)
        
    elif imputeDropdown_raw.value == "Median":
        
        imputer = SimpleImputer(strategy="median")
        imputer.fit(df_raw)
        df_raw = imputer.transform(df_raw)
        df_raw = pd.DataFrame(df_raw, columns=columns)
        
    elif imputeDropdown_raw.value == "Backward Fill":
       
        df_raw = df_raw.bfill(axis ='columns')
        
    elif imputeDropdown_raw.value == "Forward Fill":
        
        df_raw = df_raw.ffill(axis ='columns')
    
    elif imputeDropdown_raw.value == "Most Frequent":
        
        imputer = SimpleImputer(strategy="most_frequent")
        imputer.fit(df_raw)
        df_raw = imputer.transform(df_raw)
        df_raw = pd.DataFrame(df_raw, columns=columns)
    
    elif imputeDropdown_raw.value == "Custom Value":
        
        imputer = SimpleImputer(strategy="constant", fill_value=constantImputeField_raw.value)
        imputer.fit(df_raw)
        df_raw = imputer.transform(df_raw)
        df_raw = pd.DataFrame(df_raw, columns=columns)
        
    with imputeOut_raw:
        clear_output()
        print(colored(f"Missing Obs Processed!", 'green', attrs=['bold']))

In [16]:
def proceedButton2Function_raw(change):
    global df_raw
    saveButton_raw.disabled = False
    
    with histPlotOut:
        clear_output()
        hist = sns.histplot(df_raw.mean(), kde=True)
        hist.set_title("Histogram of Ensemble Data")
        hist.set_xlabel("Star Mean Magnitude")
        hist.figure.savefig("./01Data_Processing/plots/HistogramEnsemble.png")
        plt.show()

    with boxPlotOut:
        clear_output()
        box = sns.boxplot(df_raw.mean())
        box.set_title("Boxplot of Ensemble Data")
        box.set_xlabel("Star Mean Magnitude")
        box.figure.savefig("./01Data_Processing/plots/BoxplotEnsemble.png")
        plt.show()
        
    summaryPlotFunction()

In [17]:
def summaryPlotFunction():
    global df_raw
    with summaryPlotOut:
        clear_output()
        x = df_raw.mean()
        y = df_raw.std()
        c = df_raw.columns
        
        s_xs = bq.LinearScale()
        s_ys = bq.LinearScale()

        s_xax = bq.Axis(label="Star Mean Magnitude", scale=s_xs)
        s_yax = bq.Axis(label="Standard Deviation", scale=s_ys, orientation="vertical")

        summary = bq.Scatter(x=x, y=y, scales={'x': s_xs, 'y': s_ys}, names=c, display_names=False, stroke='black', unhovered_style={'opacity': 0.5}, interactions={'click': 'select'}, unselected_style={'opacity': 0.5}, selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'})

        fig = bq.Figure(title="Star Ensemble Plot (Click Stars for Details)", marks=[summary], axes=[s_xax, s_yax])
        display(fig)
        
        def click_callback(name, value):
            index = value["data"]["name"]
            with summaryPlotInfoOut:
                clear_output()
                print(colored(f"Star ID: {index}\nMean: {round(df_raw[index].mean(), 3)}\nMedian: {round(df_raw[index].median(), 3)}\nStd Dev: {round(df_raw[index].std(), 3)}\nVariance: {round(df_raw[index].var(), 3)}", 'green', attrs=['bold']))
        
        summary.on_element_click(click_callback)

In [18]:
def saveFunction(change):
    global removedColumnList
    global removedRowList
    global df_raw
    
    df_raw = df_raw.round(3)
    df_raw.to_csv("./01Data_Processing/output/ProcessedEnsemble.csv", index=None)
    
    with open(f"./01Data_Processing/logs/DP_{time.time()}.txt", "w") as f:
        print(f"{datetime.datetime.now()}", file=f)
        print(f"data: {matrixButton_raw.selected_filename}", file=f)
        print(f"columns removed: {removedColumnList}", file=f)
        print(f"rows removed: {removedRowList}", file=f)
        
    with saveOut:
        clear_output()
        print(colored(f"Processed Data and Plots Saved!", 'green', attrs=['bold']))

In [19]:
def on_change_raw():
    if imputeDropdown_raw.value == "Custom Value":
        constantImputeField_raw.disabled = False
    else:
        constantImputeField_raw.disabled = True

In [20]:
style = {'description_width': 'initial'}

# workingDirectoryButton_raw = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_raw.title = '<b>Working Directory</b>'
# workingDirectoryButton_raw.show_only_dirs = True

matrixButton_raw = ipyfilechooser.FileChooser(workingDirectory_home)
matrixButton_raw.title = '<b>Load Ensemble Data</b>'
matrixButton_raw.filter_pattern = '*.csv'

proceedButton_raw = widgets.Button(description="Load Data", button_style='warning', disabled=True)

dataInformationOut_raw = widgets.Output()

filterbox_column_raw = widgets.FloatText(description="<b>Missing Obs in Columns (%)</b>", disabled=True, style=style)
checkMissingButton_column_raw = widgets.Button(description="Detect Stars", disabled=True, style=style, button_style='success')
missingValueOut_column_raw = widgets.Output()


removeColumnBox_raw = widgets.Text(description="<b>Enter Star IDs</b>", disabled=True, style=style)
removeButton_column_raw = widgets.Button(description="Remove Stars", disabled=True, style=style, button_style='warning')
removeColumnOut_raw = widgets.Output()

filterbox_row_raw = widgets.FloatText(description="<b>Missing Obs in Row (%)</b>", disabled=True, style=style)
checkMissingButton_row_raw = widgets.Button(description="Detect Obs", disabled=True, style=style, button_style='success')
missingValueOut_row_raw = widgets.Output()
removeRowBox_raw = widgets.Text(description="<b>Enter Obs IDs</b>", disabled=True, style=style)
removeButton_row_raw = widgets.Button(description="Remove Obs", disabled=True, style=style, button_style='warning')
removeRowOut_raw = widgets.Output()

#SR --start
imputeDropdownLabel = widgets.HTML(value='<b>Impute Strategy</b>')
imputeDropdown_raw = widgets.Dropdown(options=["Custom Value", "Mean", "Median", "Backward Fill", "Forward Fill", "Most Frequent"], disabled=True, style=style)
#SR --end

#imputeDropdown_raw = widgets.Dropdown(description="<b>Impute Strategy</b>",options=["Custom Value", "Mean", "Median", "Backward Fill", "Forward Fill", "Most Frequent"], disabled=True, style=style)
constantImputeField_raw = widgets.FloatText(description="<i>Constant<i>", disabled=True)
    
imputeButton_raw = widgets.Button(description="Impute", disabled=True, style=style, icon='check', button_style='info')
imputeOut_raw = widgets.Output()

#SR --start
tableLayout = Layout(display='flex',flex_flow='column wrap',height='20vh')
misscol_table = widgets.Box(children=[missingValueOut_column_raw], layout=tableLayout)
missraw_table = widgets.Box(children=[missingValueOut_row_raw], layout=tableLayout)
#SR --end

accordian_raw = widgets.Accordion(children=[widgets.VBox([filterbox_column_raw, checkMissingButton_column_raw,misscol_table, removeColumnBox_raw, removeButton_column_raw, removeColumnOut_raw]),
                                           widgets.VBox([filterbox_row_raw, checkMissingButton_row_raw, missraw_table, removeRowBox_raw, removeButton_row_raw, removeRowOut_raw])])
accordian_raw.selected_index = None
accordian_raw.set_title(index=0, title="Remove Stars")
accordian_raw.set_title(index=1, title="Remove Obs")

proceedButton2_raw = widgets.Button(description="Plot Data", disabled=True, style=style, icon='chart-pie', button_style='primary')
histPlotOut = widgets.Output()
boxPlotOut = widgets.Output()

summaryPlotOut = widgets.Output()
summaryPlotInfoOut = widgets.Output()

saveButton_raw = widgets.Button(description="Save Data", disabled=True, icon='save', button_style='success')
saveOut = widgets.Output()

In [21]:
proceedButton_raw.on_click(dataInformation)
checkMissingButton_column_raw.on_click(checkMissingFunction_column)
removeButton_column_raw.on_click(removeMissingFunction_column)
checkMissingButton_row_raw.on_click(checkMissingFunction_row)
removeButton_row_raw.on_click(removeMissingFunction_row)
imputeButton_raw.on_click(imputeFunction)
proceedButton2_raw.on_click(proceedButton2Function_raw)
saveButton_raw.on_click(saveFunction)
imputeDropdown_raw.on_trait_change(on_change_raw)

In [22]:
#leftbox_raw = widgets.VBox([matrixButton_raw, proceedButton_raw, dataInformationOut_raw, accordian_raw, imputeDropdown_raw, constantImputeField_raw, widgets.HBox([imputeButton_raw, imputeOut_raw]), widgets.HBox([proceedButton2_raw, saveButton_raw]), saveOut])
#rightbox_raw = widgets.HBox([histPlotOut, boxPlotOut])
#upper = widgets.HBox([leftbox_raw, rightbox_raw])
#dataprocTab = widgets.VBox([upper, summaryPlotOut, summaryPlotInfoOut])

#SR 7 Aug --start
from ipywidgets import Button, GridBox, Layout, ButtonStyle

top = widgets.Box(children=[matrixButton_raw],layout=Layout(display='flex'))
left_v1 = widgets.VBox([proceedButton_raw, dataInformationOut_raw, accordian_raw])
left_v2 = widgets.VBox([imputeDropdownLabel, widgets.HBox([imputeDropdown_raw, constantImputeField_raw]), imputeButton_raw, imputeOut_raw])
left_v3 = widgets.VBox([widgets.HBox([proceedButton2_raw, saveButton_raw]), saveOut, summaryPlotInfoOut])
leftbox_raw = widgets.GridBox(children=[left_v1,left_v2,left_v3], layout=Layout(grid_gap='30px 0px'))
up_plot = widgets.Box(children=[histPlotOut, boxPlotOut],layout=Layout(display='flex', justify_content='space-around') )
rightbox_raw = widgets.VBox([up_plot, summaryPlotOut])
body = widgets.GridBox(children=[leftbox_raw, rightbox_raw],layout=Layout(grid_template_columns='30% 70%') )
dataprocTab = widgets.GridBox(children=[top,body])

In [23]:
# Variable Detection Tab

import os
import ipywidgets as widgets
import ipyfilechooser
import numpy as np
import pandas as pd
import bqplot as bq
from IPython.display import display, clear_output

In [24]:
def fileInput_ensemble():
    global df_ensemble
    global jd_ensemble
#     global directory_ensemble
    global date
    global deletedStars
    global dumpedStars
    dumpedStars = []
    deletedStars = []
#     directory_ensemble = workingDirectoryButton_ensemble.selected_path
#     os.chdir(directory_ensemble)
    df_ensemble = pd.read_csv(matrixButton_ensemble.selected_path + "/" + matrixButton_ensemble.selected_filename)
    date = pd.read_csv(jdButton_ensemble.selected_path + "/" + jdButton_ensemble.selected_filename, header=None)
    
    if "02Variable_Detection" not in os.listdir(workingDirectory_home):
        os.mkdir("02Variable_Detection")
    
    if "output" not in os.listdir("./02Variable_Detection/"):
        os.mkdir("02Variable_Detection/output")
        
    if "extras" not in os.listdir("./02Variable_Detection/"):
        os.mkdir("02Variable_Detection/extras")
        
    if "savedLCs" not in os.listdir("./02Variable_Detection/extras/"):
        os.mkdir("02Variable_Detection/extras/savedLCs")
    
    if "plots" not in os.listdir("./02Variable_Detection/"):
        os.mkdir("02Variable_Detection/plots")
    
    if "logs" not in os.listdir("./02Variable_Detection/"):
        os.mkdir("02Variable_Detection/logs")
        
    global dt
    dt = time.time()
        
    with open(f"02Variable_Detection/logs/VD_{dt}.txt", "a") as f:
        print(f"{datetime.datetime.now()}", file=f)

In [25]:
def stdFiltering():
    global df_ensemble
    global df_new_ensemble
    df_new_ensemble = df_ensemble.copy()
    deleted_columns = []
    y = 0
    while True:
        y += 1
        df_row_mean = df_new_ensemble.mean(axis=1)

        df_col_mean_1 = df_new_ensemble.mean(axis=0)

        df_row_sub = df_new_ensemble.subtract(df_row_mean, axis=0)

        df_col_mean = df_row_sub.mean(axis=0)

        df_col_sub = df_row_sub.subtract(df_col_mean, axis=1)

        df_mean = df_col_sub.mean(axis=0)

        df_std = df_col_sub.std(axis=0)

        sorted_index = []
        for i in df_new_ensemble.columns:
            if df_std[i] > stdBox_ensemble.value:
                sorted_index.append(i)
        df_sorted_index = pd.Series(df_std[sorted_index])
        if sorted_index == []:
            break
        df_new_ensemble = df_new_ensemble.drop([df_sorted_index.idxmax()], axis=1)
        deleted_columns.append(df_sorted_index.idxmax())

    df_new_ensemble = df_new_ensemble.round(3)

    aftermath = pd.DataFrame({"StarID": df_new_ensemble.columns, "MeanMagnitude": df_col_mean_1, "StdDev": df_std})
    aftermath = aftermath.round(3)

    df_new_ensemble.to_csv("02Variable_Detection/extras/NonVariables.csv", index=None)
    aftermath.to_csv("02Variable_Detection/extras/NonVariables_info.csv", index=None)

In [26]:
def ensemblePlot():
    global df_ensemble
    global df_new_ensemble
    global df_average
    global matd
    df_standard = df_new_ensemble
  
    df_average = df_ensemble.mean(axis=0)

    df_standard = df_standard.mean(axis=1)
    df_standard.to_csv("02Variable_Detection/extras/standard_row_mean.csv", index=None)

    matc = df_ensemble
    mats = df_standard

    matd = -1 * matc.subtract(mats, axis=0)
    round(matd, 3).to_csv("02Variable_Detection/extras/standard_minus_row_mean.csv", index=None)
    
    x = matd.mean()
    y = matd.std()
    
    plt.scatter(x, y, s=5, alpha=0.5)
    plt.title("Differential Ensemble plot")
    plt.xlabel("Mean Magnitude")
    plt.ylabel("Standard Deviation")
    plt.savefig("02Variable_Detection/plots/diffEnsPlot.png")
    plt.close()

    matc_col_avg = matd.mean(axis=0)
    matc_col_avg.tail()

    matg = matd.subtract(matc_col_avg, axis=1)
    matg.tail()

    matg_sq = np.square(matg)
    matg_sq.tail()

    matv_col_avg = matg_sq.mean(axis=0)
    matv_col_avg.tail()

    sd = np.sqrt(matv_col_avg)
    sd.tail()
    
    tooltip = bq.Tooltip(fields= ["index"])
    
    with ensemblePlotOut_ensemble:
        clear_output()
        x = df_average
        y = sd
        c = df_average.index
        
        lc_xs = bq.LinearScale()
        lc_ys = bq.LinearScale()
        
        lc_xax = bq.Axis(label="Julian Date", scale=lc_xs, num_ticks=3)
        lc_yax = bq.Axis(label="Magnitude", scale=lc_ys, orientation="vertical")
        
        lc_ensemble = bq.Scatter(x=date[0].values,scales={'x': lc_xs, 'y': lc_ys}, default_size=5)
        lc_ensemble_figure = bq.Figure(marks=[lc_ensemble], axes=[lc_xax, lc_yax])
        
        
        xs = bq.LinearScale()
        ys = bq.LinearScale()
        
        xax = bq.Axis(label="Star Mean Magnitude", scale=xs)
        yax = bq.Axis(label="Standard Deviation", scale=ys, orientation="vertical")
        
        ensemble = bq.Scatter(x=x, y=y, scales={'x': xs, 'y': ys}, names=c, display_names=False,  tooltip=lc_ensemble_figure, stroke='black', unhovered_style={'opacity': 0.5})

        fig = bq.Figure(title="Star Ensemble plot (Hover for Details)", marks=[ensemble], axes=[xax, yax])
        display(fig)
        
        plt.scatter(x=df_average, y=sd, s=5, alpha=0.5)
        plt.savefig("02Variable_Detection/plots/StarEnsemblePlot2.png")
        plt.close()
        
        def hover_callback(name, value):
            index = value["data"]["name"] 
            lc_ensemble_figure.title=f"Star ID {index}"
            lc_ensemble.y = df_ensemble[str(index)]
        
        ensemble.on_hover(hover_callback)

In [27]:
def processDataButtonFunction(change):
    global date
    
    with processingOut:
        clear_output()
        print(colored(f"Processing.... Please wait!!", 'blue', attrs=['bold']))
    
    fileInput_ensemble()
    if date.shape[0] == df_ensemble.shape[0]:
        stdFiltering()
        ensemblePlot()

        with processingOut:
            clear_output()
            print(colored(f"Processed!!", 'green', attrs=['bold']))
        dumpStarField_ensemble.disabled = False
        dumpStarButton_ensemble.disabled = False
        deleteLCButton_ensemble.disabled = False
    
    else:
        with processingOut:
            clear_output()
            print("Error: Different dimensions for Ensemble and Date data cannot be accepted")

In [28]:
def deleteLCFunction_ensemble(change):
    global df_ensemble
    global deletedStars
    global dt
    try:
        global df_ensemble
        x = deleteLCBox_ensemble.value
        x = x.split(",")
        column = x
        df_ensemble = df_ensemble.drop(column, axis=1)
        deletedStars.append(column)
        
        with deleteLCOut:
            clear_output()
            print(colored(f"Star ID {column} removed, Updating Plot...", 'blue', attrs=['bold']))
            
        stdFiltering()
        ensemblePlot()
        
        df_ensemble.to_csv("02Variable_Detection/extras/UpdatedEnsemble.csv", index=None)
        
        with open(f"02Variable_Detection/logs/VD_{dt}.txt", "w") as f:
            print(f"{datetime.datetime.now()}", file=f)
            print(f"deleted stars: {deletedStars}", file=f)
        
        with deleteLCOut:
            clear_output()
            print(colored(f"Plot Updated!!", 'green', attrs=['bold']))
        
    except Exception as e:
        with deleteLCOut:
            clear_output()
            print(colored("Not removed", 'red', attrs=['bold']))

In [29]:
def dumpStarFunction(change):
    global dumpedStars
    global date
    global dstar
    global dstar_matd
    dstar = pd.concat([date, df_ensemble[str(dumpStarField_ensemble.value)]],axis=1, ignore_index=True)
    dstar = dstar.round(3)
    dstar.to_csv(f"{workingDirectory_home}/02Variable_Detection/extras/savedLCs/lc_{str(dumpStarField_ensemble.value)}.csv", index=None, header=None)
        
    dstar_matd = pd.concat([date, matd[str(dumpStarField_ensemble.value)]],axis=1, ignore_index=True)
    dstar_matd = dstar_matd.round(3)
    dstar_matd.to_csv(f"{workingDirectory_home}/02Variable_Detection/output/lc_{str(dumpStarField_ensemble.value)}.csv", index=None, header=None)

    dumpedStars.append(dumpStarField_ensemble.value)
    with dumpOut:
        clear_output()
        print(colored(f"File Saved as lc_{dumpStarField_ensemble.value}.csv!", 'green', attrs=['bold']))

In [30]:
# workingDirectoryButton_ensemble = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_ensemble.title = '<b>Working Directory</b>'
# workingDirectoryButton_ensemble.show_only_dirs = True

matrixButton_ensemble = ipyfilechooser.FileChooser(workingDirectory_home)
matrixButton_ensemble.title = '<b>Load Proccessed Ensemble Data</b>'
matrixButton_ensemble.filter_pattern = '*.csv'

jdButton_ensemble = ipyfilechooser.FileChooser(workingDirectory_home)
jdButton_ensemble.title = '<b>Load Date Column</b>'
jdButton_ensemble.filter_pattern = '*.csv'

stdBox_ensemble = widgets.FloatText(description="<i>Std Dev Limit</i>", value=0.04)
processDataButton_ensemble = widgets.Button(description="Detect Variables", icon='search', button_style='info', disabled=True)
processingOut = widgets.Output()

ensemblePlotOut_ensemble = widgets.Output()
onClickLCOut_ensemble = widgets.Output()

deleteLCBox_ensemble = widgets.Text(description="<b>Delete Star</b>")
deleteLCButton_ensemble = widgets.Button(description="Delete", icon='backspace', button_style='danger', disabled=True)
deleteLCOut = widgets.Output()

#saveLCLabel = widgets.Label("Enter Star ID")
dumpStarField_ensemble = widgets.IntText(description = "<b>Enter Star ID</b>",disabled=True, style={'description_width':'initial'})
dumpStarButton_ensemble = widgets.Button(description="Save Light Curve", disabled=True, icon='save', button_style='success')
dumpOut = widgets.Output()

In [31]:
processDataButton_ensemble.on_click(processDataButtonFunction)
dumpStarButton_ensemble.on_click(dumpStarFunction)
deleteLCButton_ensemble.on_click(deleteLCFunction_ensemble)

In [32]:
#vardetectTab = widgets.VBox([matrixButton_ensemble, jdButton_ensemble, stdBox_ensemble, processDataButton_ensemble, processingOut, ensemblePlotOut_ensemble, onClickLCOut_ensemble, widgets.HBox([deleteLCBox_ensemble, deleteLCButton_ensemble]), deleteLCOut, widgets.HBox([saveLCLabel, dumpStarField_ensemble, dumpStarButton_ensemble]), dumpOut])

#SR --start
top = widgets.Box(children=[matrixButton_ensemble, jdButton_ensemble], layout=Layout(display='flex', flex_flow='column'))
leftside = widgets.VBox([stdBox_ensemble,processDataButton_ensemble, processingOut,deleteLCBox_ensemble, deleteLCButton_ensemble, deleteLCOut, dumpStarField_ensemble,dumpStarButton_ensemble, dumpOut])
rightside = widgets.VBox([ensemblePlotOut_ensemble, onClickLCOut_ensemble])
body = widgets.GridBox(children=[leftside,rightside],layout=Layout(grid_template_columns='30% 70%'))
vardetectTab = widgets.GridBox(children=[top, body])
#SR --end

In [33]:
# Detrend Data Tab

from ipywidgets import widgets
from IPython.display import display, clear_output
import ipyfilechooser
import numpy as np
import pandas as pd
import os
from statsmodels.tsa.stattools import adfuller
from scipy import signal

In [34]:
def uploadProceedFunction_dtrend(change):
#     global workingDirectory_dtrend
    global fileDirectory_dtrend
    global handleFlag 
    global checkFileName
    global handledFileName
    
    handleFlag = 0
    checkFileName = []
    handledFileName = []

    
#     workingDirectory_dtrend = workingDirectoryButton_dtrend.selected_path
    fileDirectory_dtrend = fileDirectoryButton_dtrend.selected_path
    
#     os.chdir(workingDirectory_dtrend)

    if "03Stationary_Test" not in os.listdir(workingDirectory_home):
        os.mkdir("03Stationary_Test")
        
#SR --start
    if "handleMissing" not in os.listdir("./03Stationary_Test/"):
        os.mkdir("03Stationary_Test/handleMissing")
#SR --end
        
    if "plots" not in os.listdir("./03Stationary_Test/"):
        os.mkdir("03Stationary_Test/plots")
        
    if "logs" not in os.listdir("./03Stationary_Test/"):
        os.mkdir("03Stationary_Test/logs")
        
    if "output" not in os.listdir("./03Stationary_Test/"):
        os.mkdir("03Stationary_Test/output")
    
    if "outlierRemovedLCs" not in os.listdir("./03Stationary_Test/output/"):
        os.mkdir("03Stationary_Test/output/outlierRemovedLCs")
        
    if "detrendedLCs" not in os.listdir("./03Stationary_Test/output/"):
        os.mkdir("03Stationary_Test/output/detrendedLCs")

    if "outlierRemovedComparison" not in os.listdir("./03Stationary_Test/plots/"):
        os.mkdir("03Stationary_Test/plots/outlierRemovedComparison")
        
    if "detrendedComparison" not in os.listdir("./03Stationary_Test/plots"):
        os.mkdir("03Stationary_Test/plots/detrendedComparison")
    
    #checkOutliersButton_dtrend.disabled = False
#SR --start
    checkMissingValueButton_dtrend.disabled = False
#SR --end
    
    with proceedOut_dtrend:
        clear_output()
        print(colored("Light curves loaded!!", 'green', attrs=['bold']))    

In [35]:
#SR --start
def checkMissingValueFunction(change):    
    global df
    global missFile
    global noMissFile
    
    with missingValueOut_dtrend:
        clear_output()
        print("Detecting missing values...")  
        
    fileName = []
    missCount = []   
    
    for i in os.listdir(fileDirectory_dtrend):
        if i.split(".")[-1] == "csv":
            df = pd.read_csv(fileDirectory_dtrend+'//'+i)
            df.columns = range(1, df.shape[1] + 1)
            df.index = range(1, df.shape[0] + 1)
            df = df.replace([np.inf, -np.inf, "nan "], np.nan)
            fileName.append(i)
            missCount.append(df.isnull().sum().sum())  
            
    missFile = []
    noMissFile = []
    newMissCount = []
            
    for (i,j) in zip(fileName,missCount):
        if j != 0:
            missFile.append(i)
            newMissCount.append(j)
        else:
            noMissFile.append(i)
    
    totalMiss = sum(missCount)                  
            
    if totalMiss == 0:
        with missingValueMessage:
            clear_output()
            print("No missing values...")
            imputeDropdown_dtrend.disabled = True
            handleMissingButton_dtrend.disabled = True
    else:
        pd.DataFrame({"File": missFile, "Missing values": newMissCount}).to_csv("./03Stationary_Test/logs/missingValueFiles.csv", index=None)            

        with missingValueMessage:
            clear_output()
            print(colored("Missing Values Detected!!", 'red', attrs=['bold']))
            
        with missingValueOut_dtrend:
            clear_output()
            display(pd.read_csv("./03Stationary_Test/logs/missingValueFiles.csv"))

            imputeDropdown_dtrend.disabled = False
            handleMissingButton_dtrend.disabled = False    
            
    checkOutliersButton_dtrend.disabled = False
            
#SR --end    

In [36]:
#SR --start
def imputeMissingValue(change):
    #os.chdir(fileDirectory_dtrend)
    for file in os.listdir(fileDirectory_dtrend):
        for i in noMissFile:
            if file == i:
                df1 = pd.read_csv(fileDirectory_dtrend+'//'+i)
                df1.to_csv(f"{workingDirectory_home}/03Stationary_Test/handleMissing/{i}", header=None, index = None)

    for file in os.listdir(fileDirectory_dtrend):
        for i in missFile:
            if file == i:
                df2 = pd.read_csv(fileDirectory_dtrend+'//'+i)
                columns = df2.columns
                
                if imputeDropdown_dtrend.value == "Mean":
                    imputer = SimpleImputer(strategy="mean")
                    imputer.fit(df2)
                    df2 = imputer.transform(df2)
                    df2 = pd.DataFrame(df2, columns=columns)
                    df2.to_csv(f"{workingDirectory_home}/03Stationary_Test/handleMissing/{i}", header=None,index = None)

                elif imputeDropdown_dtrend.value == "Median":
                    imputer = SimpleImputer(strategy="median")
                    imputer.fit(df2)
                    df2 = imputer.transform(df2)
                    df2 = pd.DataFrame(df2, columns=columns)
                    df2.to_csv(f"{workingDirectory_home}/03Stationary_Test/handleMissing/{i}",header=None, index = None)

    with imputeOut_dtrend:
        clear_output()
        print(colored(f"Missing Obs Processed!", 'green', attrs=['bold']))
    checkOutliersButton_dtrend.disabled = False
#SR --end   

In [37]:
# def checkOutliersFunction_dtrend(change):
    
#     global checkFileName
#     global handleFlag
    
#     with checkOutliersOut_dtrend:
#         clear_output()
#         print("Detecting Outliers...")
        
#     outliers = 0
#     detectedFile = []
#     detectedNumber = []
        
#     for i in os.listdir(fileDirectory_dtrend):
#         if i.split(".")[-1] == "csv":
#             df = pd.read_csv(fileDirectory_dtrend+"//"+i, names=["time", "mag"])
            
#             oldShape = df.shape[0]
            
#             q1, q2, q3 = np.quantile(df["mag"], [0.25, 0.5, 0.75])
#             iqr = q3 - q1
#             df = df[(df["mag"] > (q1- 1.5 * iqr))]
#             df = df[(df["mag"] < (q3 + 1.5 * iqr))]
            
#             newShape = df.shape[0]
            
#             if oldShape != newShape:
#                 outliers += 1
#                 detectedFile.append(i)
#                 detectedNumber.append(oldShape - newShape)
# #                 with open("./03Stationary_Test/logs/outliers.txt", "a") as f:
# #                     print(f"file: {i}, outliers: {oldShape - newShape}", file=f)
                    
#             checkFileName.append(i)
    
#     pd.DataFrame({"file": detectedFile, "outliers": detectedNumber}).to_csv("./03Stationary_Test/logs/outliers.csv", index=None)
                    
#     with checkOutliersOut_dtrend:
#         clear_output()
#         if outliers != 0:
#             handleOutliersButton_dtrend.disabled = False
#             print(colored("Outliers Detected!!", 'red', attrs=['bold']))
#             display(pd.read_csv("./03Stationary_Test/logs/outliers.csv"))
#             handleFlag = 1
#         else:
#             adfTestButton_dtrend.disabled = False
#             print(colored("No Outliers present!!", 'green', attrs=['bold']))
            
            
    

In [38]:
#SR --start
def checkOutliersFunction_dtrend(change): 
    global outFiles
    global noOutFiles
    #global detectedFiles
    #global handleFlag
    global checkFileName
    checkFileName = []

    with checkOutliersOut_dtrend:
        clear_output()
        print("Detecting Outliers...")
        
    #outliers = 0
    #detectedFiles = []
    outliersCount = []
    outFiles = []
    noOutFiles = []
    
    if missFile != []:
        path = workingDirectory_home+"/03Stationary_Test/handleMissing/"
    elif missFile == []:
        path = fileDirectory_dtrend
    else:
        pass
    
    loadedFiles = missFile + noMissFile 
    print(loadedFiles)
    
    for file in os.listdir(path):
        for i in loadedFiles:
            if file == i:
                if i.split(".")[-1] == "csv":
                    df = pd.read_csv(path+'//'+i, names=["time", "mag"])
                    oldShape = df.shape[0]
                    q1, q2, q3 = np.quantile(df["mag"], [0.25, 0.5, 0.75])
                    iqr = q3 - q1
                    df = df[(df["mag"] > (q1- 1.5 * iqr))]
                    df = df[(df["mag"] < (q3 + 1.5 * iqr))]
                    newShape = df.shape[0]
                    
                    if oldShape != newShape:
                        outFiles.append(i)
                        outliersCount.append(oldShape - newShape)
                    else:
                        noOutFiles.append(i)

                    checkFileName.append(i)
    
    pd.DataFrame({"File":outFiles, "Outliers": outliersCount}).to_csv("./03Stationary_Test/logs/outliers.csv", index=None)
    
    if outFiles != []:
        with checkOutliersMessage:
            clear_output()
            print(colored("Outliers Detected!!", 'red', attrs=['bold']))
            
        with checkOutliersOut_dtrend:
            clear_output()
            handleOutliersButton_dtrend.disabled = False
            display(pd.read_csv("./03Stationary_Test/logs/outliers.csv"))
            
    else:
        with checkOutliersMessage:
            clear_output()
            handleOutliersButton_dtrend.disabled = True
            adfTestButton_dtrend.disabled = False
            print(colored("No Outliers present!!", 'green', attrs=['bold']))
            
#     with checkOutliersOut_dtrend:
#         clear_output()
#         if outFiles != []:
#             handleOutliersButton_dtrend.disabled = False
#             print(colored("Outliers Detected!!", 'red', attrs=['bold']))
#             display(pd.read_csv("./03Stationary_Test/logs/outliers.csv"))
#         else:
#             handleOutliersButton_dtrend.disabled = True
#             adfTestButton_dtrend.disabled = False
#             print(colored("No Outliers present!!", 'green', attrs=['bold']))
#SR --end    

In [39]:
# def handleOutliersFunction_dtrend(change):
#     with handleOutliersOut_dtrend:
#         clear_output()
#         print(colored("Removing Outliers, Please Wait...", 'blue', attrs=['bold']))
     
#     global handledFileName
            
#     for i in os.listdir(fileDirectory_dtrend):
#         if i.split(".")[-1] == "csv":
#             df = pd.read_csv(fileDirectory_dtrend+"//"+i, names=["time", "mag"])

#             plt.subplot(1, 2, 1)
#             plt.scatter(df["time"], df["mag"])
#             plt.title("Original LC")

#             q1, q2, q3 = np.quantile(df["mag"], [0.25, 0.5, 0.75])
#             iqr = q3 - q1
#             df = df[(df["mag"] > (q1- 1.5 * iqr))]
#             df = df[(df["mag"] < (q3 + 1.5 * iqr))]
#             df.to_csv(f"{workingDirectory_home}/03Stationary_Test/output/outlierRemovedLCs/{i}", index=None, header=None)

#             plt.subplot(1, 2, 2)
#             plt.scatter(df["time"], df["mag"])

#             plt.title("Outliers Removed")
#             plt.savefig(f"{workingDirectory_home}/03Stationary_Test/plots/outlierRemovedComparison/{i.split('.csv')[0]}_comparison.png")
#             plt.close()

#             handledFileName.append(i)

#     with handleOutliersOut_dtrend:
#         clear_output()
#         print(colored("Outliers Removed!!", 'green', attrs=['bold']))
        
#     adfTestButton_dtrend.disabled = False

In [40]:
#SR --start
def handleOutliersFunction_dtrend(change):
    with handleOutliersOut_dtrend:
        clear_output()
        print(colored("Removing Outliers, Please Wait...", 'blue', attrs=['bold']))
    
    if missFile != []:
        path = workingDirectory_home+"/03Stationary_Test/handleMissing/"
    elif missFile == []:
        path = fileDirectory_dtrend
    else:
        pass
        
    for file in os.listdir(path):
        for i in noOutFiles:
            if file == i:
                if i.split(".")[-1] == "csv":
                    df = pd.read_csv(path+'//'+i, names=["time", "mag"])
                    df.to_csv(f"{workingDirectory_home}/03Stationary_Test/output/outlierRemovedLCs/{i}", index=None, header=None)
                
        for i in outFiles:
            if file == i:
                if i.split(".")[-1] == "csv":
                    df = pd.read_csv(path+'//'+i, names=["time", "mag"])

                    plt.subplot(1, 2, 1)
                    plt.scatter(df["time"], df["mag"])
                    plt.title("Original LC")

                    q1, q2, q3 = np.quantile(df["mag"], [0.25, 0.5, 0.75])
                    iqr = q3 - q1
                    df = df[(df["mag"] > (q1- 1.5 * iqr))]
                    df = df[(df["mag"] < (q3 + 1.5 * iqr))]
                    df.to_csv(f"{workingDirectory_home}/03Stationary_Test/output/outlierRemovedLCs/{i}", index=None, header=None)

                    plt.subplot(1, 2, 2)
                    plt.scatter(df["time"], df["mag"])

                    plt.title("Outliers Removed")
                    plt.savefig(f"{workingDirectory_home}/03Stationary_Test/plots/outlierRemovedComparison/{i.split('.csv')[0]}_comparison.png")
                    plt.close()

    with handleOutliersOut_dtrend:
        clear_output()
        print(colored("Outliers Removed!!", 'green', attrs=['bold']))
        
    adfTestButton_dtrend.disabled = False
#SR --end

In [41]:
# def adfTestFunction_dtrend(change):
    
#     global handleFlag
#     global handledFileName
#     global checkFileName
    
    
#     with adfInfoOut:
#         clear_output()
#         print(colored("Processing...", 'blue', attrs=['bold']))
        
#     global adfResult
    
#     data = []
#     testStats = []
#     p = []
#     critical1 = []
#     critical5 = []
#     critical10 = []
#     stationary = []
                    
#     if handleFlag == 0:
#         for i in checkFileName:
#             df = pd.read_csv(fileDirectory_dtrend+"//"+i, names=["time", "mag"])
#             df_sort = df.sort_values(by="time")

#             test = adfuller(df_sort["mag"], autolag="AIC")

#             data.append(i)
#             testStats.append(test[0])
#             p.append(test[1])
#             critical1.append(test[4]["1%"])
#             critical5.append(test[4]["5%"])
#             critical10.append(test[4]["10%"])

#     #         if (test[0] < test[4]["1%"]) and (test[0] < test[4]["5%"]) and (test[0] < test[4]["10%"]) and (test[1] < 0.05):
#     #             stationary.append(1)
#     #         else:
#     #             stationary.append(0)

#             if (test[1] < 0.05):
#                 stationary.append(1)
#             elif (test[1] > 0.05):
#                 if (test[0] < test[4]["1%"]) and (test[0] < test[4]["5%"]) and (test[0] < test[4]["10%"]):
#                     stationary.append(1)
#                 else:
#                     stationary.append(0)
    
#     elif handleFlag == 1:
#         for i in handledFileName:
#             df = pd.read_csv(workingDirectory_home+"/03Stationary_Test/output/outlierRemovedLCs/"+i, names=["time", "mag"])
#             df_sort = df.sort_values(by="time")

#             test = adfuller(df_sort["mag"], autolag="AIC")

#             data.append(i)
#             testStats.append(test[0])
#             p.append(test[1])
#             critical1.append(test[4]["1%"])
#             critical5.append(test[4]["5%"])
#             critical10.append(test[4]["10%"])

#     #         if (test[0] < test[4]["1%"]) and (test[0] < test[4]["5%"]) and (test[0] < test[4]["10%"]) and (test[1] < 0.05):
#     #             stationary.append(1)
#     #         else:
#     #             stationary.append(0)

#             if (test[1] < 0.05):
#                 stationary.append(1)
#             elif (test[1] > 0.05):
#                 if (test[0] < test[4]["1%"]) and (test[0] < test[4]["5%"]) and (test[0] < test[4]["10%"]):
#                     stationary.append(1)
#                 else:
#                     stationary.append(0)
    
                

#     adfResult = pd.DataFrame({"File Name": data, 
#                               "ADF Score": testStats, 
#                               "p-value": p, 
#                               "Critical_value_1%": critical1, 
#                               "Critical_value_5%": critical5, 
#                               "Critical_value_10%": critical10, 
#                               "Stationary": stationary})
    
#     adfResult.to_csv("./03Stationary_Test/output/adfResult.csv", index=None)
    
#     for i in stationary:
#         if i == 0:
#             dtrendButton_dtrend.disabled = False
    
#     with adfInfoOut:
#         clear_output()
#         print(colored("Stationarity Checked!!\n\n", 'green', attrs=['bold']))
#         print(colored("ADF Statistics [0=False, 1=True]", 'red', attrs=['bold','reverse']))
#         display(adfResult)


In [42]:
#SR --start
def adfTestFunction_dtrend(change):
    with adfInfoOut:
        clear_output()
        print(colored("Processing...", 'blue', attrs=['bold']))
        
    global adfResult
    
    data = []
    testStats = []
    p = []
    critical1 = []
    critical5 = []
    critical10 = []
    stationary = []
    
    if outFiles == []:
        if missFile != []:
            path = workingDirectory_home+"/03Stationary_Test/handleMissing/"
        else:
            path = fileDirectory_dtrend
    else:
        path = workingDirectory_home+"/03Stationary_Test/output/outlierRemovedLCs/"
                    
    allFile = outFiles + noOutFiles
    
    for file in os.listdir(path):
        for i in allFile:
            if file == i:
                df = pd.read_csv(path+'//'+i, names=["time", "mag"])
                df_sort = df.sort_values(by="time")

                test = adfuller(df_sort["mag"], autolag="AIC")

                data.append(i)
                testStats.append(test[0])
                p.append(test[1])
                critical1.append(test[4]["1%"])
                critical5.append(test[4]["5%"])
                critical10.append(test[4]["10%"])

                if (test[1] < 0.05):
                    stationary.append(1)
                elif (test[1] > 0.05):
                    if (test[0] < test[4]["1%"]) and (test[0] < test[4]["5%"]) and (test[0] < test[4]["10%"]):
                        stationary.append(1)
                    else:
                        stationary.append(0)  
                
    adfResult = pd.DataFrame({"File Name": data, 
                              "ADF Score": testStats, 
                              "p-value": p, 
                              "Critical_value_1%": critical1, 
                              "Critical_value_5%": critical5, 
                              "Critical_value_10%": critical10, 
                              "Stationary": stationary})
    
    adfResult.to_csv("./03Stationary_Test/output/adfResult.csv", index=None)
    
    c = 0
    for i in stationary:
        if i == 0:
            c = c+1
        else:
            pass
            
    if c > 0:
        dtrendButton_dtrend.disabled = False
    else:
        dtrendButton_dtrend.disabled = True
        
        with dtrendInfoOut:
            clear_output()
        
    
    with adfMessageOut:
        clear_output()
        print(colored("Stationarity Checked!!\n\n", 'green', attrs=['bold']))
        
    with adfInfoOut:
        clear_output()
        print(colored("ADF Statistics [0=False, 1=True]", 'red', attrs=['bold','reverse']))
        display(adfResult)
#SR --end        

In [43]:
# def dtrendFunction_dtrend(change):
#     with dtrendInfoOut:
#         clear_output()
#         print(colored("Detrending data...", 'blue', attrs=['bold']))  
        
#     for i in range(adfResult.shape[0]):
#         df = pd.read_csv(path+'//'+adfResult.iloc[i]["File Name"], names=["time", "mag"])
        
#         plt.subplot(1, 2, 1)
#         plt.scatter(df["time"], df["mag"])
#         plt.title("Original LC")

        
#         df_sort = df.sort_values(by="time")
#         if adfResult.iloc[i]["Stationary"] == 0:
#             df_sort["dtrended"] = signal.detrend(df_sort["mag"])
#             df_sort = df_sort.drop(["mag"], axis=1)
            
#             plt.subplot(1, 2, 2)
#             plt.scatter(df_sort["time"], df_sort["dtrended"])
#             plt.title("Detrended LC")
#         df_sort.to_csv(f"{workingDirectory_home}/03Stationary_Test/output/detrendedLCs/{adfResult.iloc[i]['File Name']}", index=None, header=None)
        
        
#         plt.savefig(f"{workingDirectory_home}/03Stationary_Test/plots/detrendedComparison/{adfResult.iloc[i]['File Name'].split('.csv')[0]}_comparison.png")
#         plt.close()
        
#     with dtrendInfoOut:
#         clear_output()
#         print(colored("Light Curves Detrended!!", 'green', attrs=['bold']))

In [44]:
#SR --start
def dtrendFunction_dtrend(change):
    with dtrendInfoOut:
        clear_output()
        print(colored("Detrending data...", 'blue', attrs=['bold']))
             
    if outFiles == []:
        if missFile != []:
            path = workingDirectory_home+"/03Stationary_Test/handleMissing/"
        else:
            path = fileDirectory_dtrend
    else:
        path = workingDirectory_home+"/03Stationary_Test/output/outlierRemovedLCs/"      
        
    for i in range(adfResult.shape[0]):
        df = pd.read_csv(path+'//'+adfResult.iloc[i]["File Name"], names=["time", "mag"])        
        df_sort = df.sort_values(by="time")
        df_sort.to_csv(f"{workingDirectory_home}/03Stationary_Test/output/detrendedLCs/{adfResult.iloc[i]['File Name']}", index=None, header=None)
        
        if adfResult.iloc[i]["Stationary"] == 0:
            plt.subplot(1, 2, 1)
            plt.scatter(df["time"], df["mag"])
            plt.title("Original LC")
            
            df_sort["dtrended"] = signal.detrend(df_sort["mag"])
            df_sort = df_sort.drop(["mag"], axis=1)
            
            plt.subplot(1, 2, 2)
            plt.scatter(df_sort["time"], df_sort["dtrended"])
            plt.title("Detrended LC")
              
            plt.savefig(f"{workingDirectory_home}/03Stationary_Test/plots/detrendedComparison/{adfResult.iloc[i]['File Name'].split('.csv')[0]}_comparison.png")
            plt.close()
        
    with dtrendInfoOut:
        clear_output()
        print(colored("Light Curves Detrended!!", 'green', attrs=['bold']))
        
#SR --end

In [45]:
# workingDirectoryButton_dtrend = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_dtrend.title = '<b>Working Directory</b>'
# workingDirectoryButton_dtrend.show_only_dirs = True

fileDirectoryButton_dtrend = ipyfilechooser.FileChooser(workingDirectory_home)
fileDirectoryButton_dtrend.title = '<b>Select Light Curve Directory</b>'

uploadProceedButton_dtrend = widgets.Button(description="Load Data", button_style='warning', disabled=True)
proceedOut_dtrend = widgets.Output()

#SR --start
checkMissingValueButton_dtrend = widgets.Button(description="Check Missing Values", disabled=True, icon='search', button_style='info')
missingValueMessage = widgets.Output()
missingValueOut_dtrend = widgets.Output()

imputeDropdownLabel_dtrend = widgets.HTML(value='<b>Impute Strategy</b>')
imputeDropdown_dtrend = widgets.Dropdown(options=["Mean", "Median"], disabled=True, style=style)
handleMissingButton_dtrend = widgets.Button(description="Handle Missing Values", disabled=True, style=style, icon='check', button_style='success')
imputeOut_dtrend = widgets.Output()
#SR --end

checkOutliersButton_dtrend = widgets.Button(description="Check Outliers", disabled=True, icon='search', button_style='info')
#SR --start
checkOutliersMessage = widgets.Output()
#SR --end
checkOutliersOut_dtrend = widgets.Output()

handleOutliersButton_dtrend = widgets.Button(description="Remove Outliers", disabled=True, icon='eraser', button_style='danger')
handleOutliersOut_dtrend = widgets.Output()

adfTestButton_dtrend = widgets.Button(description="Check Stationarity", disabled=True, icon='search', button_style='info')
adfMessageOut = widgets.Output()
adfInfoOut = widgets.Output()

dtrendButton_dtrend = widgets.Button(description="Detrend Data", disabled=True, icon='location-arrow', button_style='success')
dtrendInfoOut = widgets.Output()

In [46]:
uploadProceedButton_dtrend.on_click(uploadProceedFunction_dtrend)

#SR --start
checkMissingValueButton_dtrend.on_click(checkMissingValueFunction)
handleMissingButton_dtrend.on_click(imputeMissingValue)
#SR --end

checkOutliersButton_dtrend.on_click(checkOutliersFunction_dtrend)
handleOutliersButton_dtrend.on_click(handleOutliersFunction_dtrend)
adfTestButton_dtrend.on_click(adfTestFunction_dtrend)
dtrendButton_dtrend.on_click(dtrendFunction_dtrend)

In [47]:
#stationaryTab = widgets.VBox([fileDirectoryButton_dtrend, uploadProceedButton_dtrend, proceedOut_dtrend, checkOutliersButton_dtrend, checkOutliersOut_dtrend, handleOutliersButton_dtrend, handleOutliersOut_dtrend, adfTestButton_dtrend, adfInfoOut, dtrendButton_dtrend, dtrendInfoOut])
#SR --start
top = widgets.Box(children=[fileDirectoryButton_dtrend], layout=Layout(display='flex'))
left = widgets.VBox([uploadProceedButton_dtrend, proceedOut_dtrend, checkMissingValueButton_dtrend, missingValueMessage, imputeDropdownLabel_dtrend, imputeDropdown_dtrend, handleMissingButton_dtrend, imputeOut_dtrend, checkOutliersButton_dtrend, checkOutliersMessage, handleOutliersButton_dtrend, handleOutliersOut_dtrend, adfTestButton_dtrend, adfMessageOut, dtrendButton_dtrend,dtrendInfoOut])
missTable = widgets.Box(children=[missingValueOut_dtrend], layout=Layout(display='flex', height='40vh'))
outTable = widgets.Box(children=[checkOutliersOut_dtrend], layout=Layout(display='flex', height='40vh'))
mid = widgets.VBox([missTable, outTable],)
right = widgets.Box(children = [adfInfoOut], layout=Layout(display='flex', flex_flow='column', height='80vh'))
body = widgets.GridBox(children = [left,mid,right], layout=Layout(grid_template_columns='20% 15% 65% '))
stationaryTab = widgets.GridBox(children = [top,body])
#SR --end

In [48]:
# Period Detection Tab

import os
import ipywidgets as widgets
import ipyfilechooser
import pandas as pd
import numpy as np
from gatspy.periodic import LombScargle, LombScargleFast
from features.modelFeatures import falseProbAlarm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('seaborn-ticks')
import math
from IPython.display import display, clear_output

In [49]:
def uploadFunction(change):
#     global directory_periodogram
    global fileName_periodogram
    global lcFolder_periodogram
#     directory_periodogram = workingDirectoryButton_periodogram.selected_path
#     os.chdir(directory_periodogram)
    
    global df_periodogram
    fileName_periodogram = uploadButton_periodogram.selected_path+"/"+uploadButton_periodogram.selected_filename
    df_periodogram = pd.read_csv(fileName_periodogram, names=["JD", "Mag"])
    
    lcFolder_periodogram = f"{uploadButton_periodogram.selected_filename.split('.csv')[0]}"
    
    if "04Period_Detection" not in os.listdir(workingDirectory_home):
        os.mkdir("04Period_Detection")
    
    if lcFolder_periodogram not in os.listdir("./04Period_Detection/"):
        os.mkdir(f"04Period_Detection/{lcFolder_periodogram}")
    
    if "logs" not in os.listdir(f"./04Period_Detection/{lcFolder_periodogram}/"):
        os.mkdir(f"04Period_Detection/{lcFolder_periodogram}/logs")

    if "output" not in os.listdir(f"./04Period_Detection/{lcFolder_periodogram}/"):
        os.mkdir(f"04Period_Detection/{lcFolder_periodogram}/output")

    if "plots" not in os.listdir(f"./04Period_Detection/{lcFolder_periodogram}/"):
        os.mkdir(f"04Period_Detection/{lcFolder_periodogram}/plots")
    
    plotLCFunction()

    algorithmDropdown_periodogram.disabled = False
    algorithmProceedButton_periodogram.disabled = False

In [50]:
#SR --start
# from tqdm.notebook import tqdm, trange
# import time  
from tqdm import tqdm_notebook as tqdm
#SR --end

In [51]:
def algorithmFunction(change):
    if algorithmDropdown_periodogram.value == "Custom":
        custom()
    elif algorithmDropdown_periodogram.value == "LombScargle" or algorithmDropdown_periodogram.value == "LombScargleFast":
        lombScarglePeriod()

In [52]:
def custom():
    t0Box_periodogram.value = t0max()
    periodBox_periodogram.value = 1
    minimumBox_periodogram.disabled = True
    maximumBox_periodogram.disabled = True
    dyBox_periodogram.disabled = True
    periodBox_periodogram.disabled = False
    calculatePeriodButton_periodogram.disabled = True
    calculatePCButton_periodogram.disabled = False

In [53]:
def lombScarglePeriod():
    minimumBox_periodogram.disabled = False
    maximumBox_periodogram.disabled = False
    t0Box_periodogram.disabled = False
    dyBox_periodogram.disabled = False
    periodBox_periodogram.disabled = True
    t0Box_periodogram.value = t0max()
    calculatePeriodButton_periodogram.disabled = False
    calculatePCButton_periodogram.disabled = True

In [54]:

def t0max():

    for i,j in zip(df_periodogram[df_periodogram.columns[0]],df_periodogram[df_periodogram.columns[1]]):
        if j == max(df_periodogram[df_periodogram.columns[1]]):
            t0 = i

    return t0

In [55]:
def plotLCFunction():
    with LCPlotOut_periodogram:
        clear_output()
        plt.scatter(df_periodogram[df_periodogram.columns[0]], df_periodogram[df_periodogram.columns[1]], s=5, alpha=0.5)
        plt.xlabel(df_periodogram.columns[0])
        plt.ylabel(df_periodogram.columns[1])
        plt.gca().invert_yaxis()
        plt.title("Original Light Curve")
        plt.savefig(f"04Period_Detection/{lcFolder_periodogram}/plots/{uploadButton_periodogram.selected_filename.split('.csv')[0]}_OrgLC_Plot.png")
        plt.show()

In [56]:
def calculatePhaseFunction(change):
    global phaseDf_periodogram
    phaseDf_periodogram = df_periodogram.values
    global m
    m = []
    for i in phaseDf_periodogram[:, 0]:  
        x = i - t0Box_periodogram.value
        m.append((x / periodBox_periodogram.value) - math.floor(x / periodBox_periodogram.value))

    phaseDf_periodogram = pd.DataFrame({"Phase": m, "Mag": phaseDf_periodogram[:, 1]}, )
    phaseDf_periodogram = phaseDf_periodogram.round(3)
    phaseDf_periodogram.to_csv(f"04Period_Detection/{lcFolder_periodogram}/output/{uploadButton_periodogram.selected_filename.split('.csv')[0]}_Phased_LC.csv", index=False)
    
    plotPC()
    
    with open(f"04Period_Detection/{lcFolder_periodogram}/logs/{uploadButton_periodogram.selected_filename.split('.csv')[0]}_log.txt", "w") as f:
        print(f"Min Period: {minimumBox_periodogram.value}", file=f)
        print(f"Max Period: {maximumBox_periodogram.value}", file=f)
        print(f"Epoch (t0): {t0Box_periodogram.value}", file=f)
        print(f"Std Dev: {dyBox_periodogram.value}", file=f)
        print(f"Period: {periodBox_periodogram.value}", file=f)
        print(f"False Alarm Probability: {falseProbAlarm(df_periodogram[df_periodogram.columns[0]], df_periodogram[df_periodogram.columns[1]])}", file=f)

In [57]:
def plotPC():
    with PCPlotOut_periodogram:
        clear_output()
        plt.scatter(phaseDf_periodogram[phaseDf_periodogram.columns[0]], phaseDf_periodogram[phaseDf_periodogram.columns[1]], s=5, alpha=0.5)
        plt.xlabel(phaseDf_periodogram.columns[0])
        plt.ylabel(phaseDf_periodogram.columns[1])
        plt.gca().invert_yaxis()
        plt.title("Phased Light Curve")
        plt.savefig(f"04Period_Detection/{lcFolder_periodogram}/plots/{uploadButton_periodogram.selected_filename.split('.csv')[0]}_PhasedLC_Plot.png")
        plt.show()

In [58]:
def calculatePeriodFunction(change):
    global originalPeriod
    try:
        with calculatePeriodOut_periodogram:
            clear_output()
            print(colored("Calculating Period...", 'blue', attrs=['bold']))
#SR --start
            for i in trange(300):
                time.sleep(0.07)
#SR --end
            
        if algorithmDropdown_periodogram.value == "LombScargleFast":
        
            gatspy_period = LombScargleFast()
            gatspy_period.fit(df_periodogram[df_periodogram.columns[0]], df_periodogram[df_periodogram.columns[1]], dyBox_periodogram.value)
            gatspy_period.optimizer.period_range=(minimumBox_periodogram.value, maximumBox_periodogram.value)
            originalPeriod = gatspy_period.best_period
            periodBox_periodogram.value = originalPeriod

            generatedPeriod_periodogram = np.linspace(minimumBox_periodogram.value, maximumBox_periodogram.value, 10000)
            score = gatspy_period.score(generatedPeriod_periodogram)
        
        elif algorithmDropdown_periodogram.value == "LombScargle":
            
            gatspy_period = LombScargle()
            gatspy_period.fit(df_periodogram[df_periodogram.columns[0]], df_periodogram[df_periodogram.columns[1]], dyBox_periodogram.value)
            gatspy_period.optimizer.period_range=(minimumBox_periodogram.value, maximumBox_periodogram.value)
            originalPeriod = gatspy_period.best_period
            periodBox_periodogram.value = originalPeriod

            generatedPeriod_periodogram = np.linspace(minimumBox_periodogram.value, maximumBox_periodogram.value, 10000)
            score = gatspy_period.score(generatedPeriod_periodogram)

        with powerPlotOut:
            clear_output()
            plt.plot(generatedPeriod_periodogram, score)
            plt.xlabel("Period")
            plt.ylabel("Lomb-Scargle Power")
            plt.title("Lomb-Scargle periodogram as a function of period")
            plt.savefig(f"04Period_Detection/{lcFolder_periodogram}/plots/{uploadButton_periodogram.selected_filename.split('.csv')[0]}_Power_Plot.png")
            plt.show()

        resetPeriodButton.disabled = False
        doublePeriodButton.disabled = False
        halfPeriodButton.disabled = False
        calculatePeriodButton_periodogram.disabled = False
        calculatePCButton_periodogram.disabled = False
        
        with calculatePeriodOut_periodogram:
            clear_output()
            print(colored(f"False Alarm Probability: {falseProbAlarm(df_periodogram[df_periodogram.columns[0]], df_periodogram[df_periodogram.columns[1]])}", 'green', attrs=['bold']))
    
    except Exception as e:
        with calculatePeriodOut_periodogram:
            clear_output()
            print(colored("The optimizer is not designed to search for periods larger than the data baseline.", 'red', attrs=['bold']))

In [59]:
def resetPeriodButtonFunction(change):
    periodBox_periodogram.value = originalPeriod

In [60]:
def doublePeriodButtonFunction(change):
    periodBox_periodogram.value = originalPeriod * 2

In [61]:
def halfPeriodButtonFunction(change):
    periodBox_periodogram.value = originalPeriod * 0.5

In [62]:
# workingDirectoryButton_periodogram = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_periodogram.title = '<b>Working Directory</b>'

uploadButton_periodogram = ipyfilechooser.FileChooser(workingDirectory_home)
uploadButton_periodogram.title = '<b>Load Light Curve</b>'
uploadButton_periodogram.filter_pattern = '*.csv'

uploadProceedButton_periodogram = widgets.Button(description="Load Data", button_style='warning', disabled=True)

algorithmDropdown_periodogram = widgets.Dropdown(options=["Custom", "LombScargle", "LombScargleFast"], value="LombScargle", description='<i>Algorithm</i>', disabled=True)
minimumBox_periodogram = widgets.FloatText(value=0.1, description="<i>Min Period</i>", disabled=True)
maximumBox_periodogram = widgets.FloatText(value=10, description="<i>Max Period</i>", disabled=True)
t0Box_periodogram = widgets.FloatText(value=0, description="<i>Epoch (t0)</i>", disabled=True)
dyBox_periodogram = widgets.FloatText(value=0.1, description="<i>Std Dev</i>", disabled=True)
periodBox_periodogram = widgets.FloatText(value=10, description="<b>Period</b>", disabled=True)
calculatePeriodButton_periodogram = widgets.Button(description="Calculate Period", icon='search', button_style='danger', disabled=True)
calculatePeriodOut_periodogram = widgets.Output()
space1 = widgets.Label("Modify Period")
doublePeriodButton = widgets.Button(description="Double Period", disabled=True, button_style='primary')
halfPeriodButton = widgets.Button(description="Half Period", disabled=True, button_style='success')
resetPeriodButton = widgets.Button( disabled=True, icon='sync', button_style='warning', layout=Layout(width="10%"))
space2 = widgets.Label("")
calculatePCButton_periodogram = widgets.Button(description="Plot Phased LC", disabled=True, icon='chart-area', button_style='info')

LCPlotOut_periodogram = widgets.Output()
PCPlotOut_periodogram = widgets.Output()
powerPlotOut = widgets.Output()

In [63]:
uploadProceedButton_periodogram.on_click(uploadFunction)
calculatePeriodButton_periodogram.on_click(calculatePeriodFunction)
resetPeriodButton.on_click(resetPeriodButtonFunction)
doublePeriodButton.on_click(doublePeriodButtonFunction)
halfPeriodButton.on_click(halfPeriodButtonFunction)
calculatePCButton_periodogram.on_click(calculatePhaseFunction)
algorithmDropdown_periodogram.on_trait_change(algorithmFunction)

In [64]:
# leftBox_periodogram = widgets.VBox([uploadButton_periodogram, uploadProceedButton_periodogram, algorithmDropdown_periodogram, minimumBox_periodogram, maximumBox_periodogram, t0Box_periodogram, dyBox_periodogram, periodBox_periodogram, widgets.HBox([calculatePeriodButton_periodogram, calculatePCButton_periodogram]), calculatePeriodOut_periodogram, space1, widgets.HBox([doublePeriodButton, halfPeriodButton, resetPeriodButton]), space2, powerPlotOut])
# rightBox_periodogram = widgets.VBox([LCPlotOut_periodogram, PCPlotOut_periodogram])

# perioddetectTab = widgets.HBox([leftBox_periodogram, rightBox_periodogram])

#SR --start
top = widgets.Box(children=[uploadButton_periodogram], layout=Layout(display='flex'))
left = widgets.VBox([ uploadProceedButton_periodogram, algorithmDropdown_periodogram, minimumBox_periodogram, maximumBox_periodogram, t0Box_periodogram, dyBox_periodogram, periodBox_periodogram, calculatePeriodOut_periodogram, widgets.HBox([calculatePeriodButton_periodogram, calculatePCButton_periodogram]),  space1, widgets.HBox([doublePeriodButton, halfPeriodButton, resetPeriodButton]), space2])
right =  widgets.GridBox(children=[LCPlotOut_periodogram, powerPlotOut, PCPlotOut_periodogram], layout=Layout(grid_template_columns='50% 50% ',grid_template_rows='50% 50%',))
body = widgets.GridBox(children=[left,right], layout=Layout(grid_template_columns='30% 70% '))
perioddetectTab=widgets.GridBox(children=[top,body])
#SR --end

In [65]:
# Feature Extraction Tab

from ipywidgets import widgets
from IPython.display import display, clear_output
import ipyfilechooser
import numpy as np
import pandas as pd
import os
from features import allFeatures, modelFeatures

In [66]:
def uploadProceedFunction_feature(change):

#     global workingDirectory_feature
    global fileDirectory_feature

#     workingDirectory_feature = workingDirectoryButton_feature.selected_path
    fileDirectory_feature = fileDirectoryButton_feature.selected_path
    
#     os.chdir(workingDirectory_feature)
        
    if "05Feature_Detection" not in os.listdir(workingDirectory_home):
        os.mkdir("05Feature_Detection")
        
    if "output" not in os.listdir("./05Feature_Detection/"):
        os.mkdir("05Feature_Detection/output")

    if dataSourceDropdown_feature.value == "ASAS":
        cleanButton_feature.disabled = False
        saveCleanedDataButton_feature = False
        selectFeaturesDropdown_feature.disabled = False
        extractFeaturesButton_feature.disabled = False
    if dataSourceDropdown_feature.value == "Standard":
        cleanButton_feature.disabled = True
        saveCleanedDataButton_feature = True
        selectFeaturesDropdown_feature.disabled = False
        extractFeaturesButton_feature.disabled = False
    
    with uploadInfoOut:
        clear_output()
        print(colored("Light Curves Loaded!!", 'green', attrs=['bold']))

In [67]:
def dataCleaning_feature(x):
    list1=[]
    list2=[]
    list3=[]
    list4=[]
    for line in x:
        li=line.strip()
        if not li.startswith("#"):
            l=line.rstrip()
            a=l.split()
            b=a[0]
            list1.append(b)
            c=a[1]
            list2.append(c)
            d=a[6]
            list3.append(d)
            e=a[-2]
            list4.append(e)
    df=pd.DataFrame({'TIME':list1,'VALUE':list2,'MER_3':list3,'GRADE':list4})
    df.drop(df.loc[df['GRADE']=='C'].index, inplace=True)
    df.drop(df.loc[df['GRADE']=='D'].index, inplace=True)
    df.drop(['GRADE', 'MER_3'], axis = 1, inplace=True)
    return df

In [68]:
def dataExtraction_feature(df):
    if selectFeaturesDropdown_feature.value == "Model Features":
        features = []

        maxRange = 1000

        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 1000:
            maxRange = 800

        if (max(df[df.columns[df.columns[0]]]) - min(df[df.columns[0]])) < 800:
            maxRange = 600
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 600:
            maxRange = 400
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 400:
            maxRange = 200
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 200:
            maxRange = 100

        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 100:
            maxRange = 10

        gatspy_period = LombScargleFast()
        gatspy_period.fit(df[df.columns[0]], df[df.columns[1]], 0.01)
        gatspy_period.optimizer.period_range=(0.1, maxRange)
        originalPeriod = gatspy_period.best_period 
        

        autocorr = modelFeatures.autocorr(df[df.columns[1]])
        ecdf = modelFeatures.ecdf(df[df.columns[1]], d=1)[0]
        entropy = modelFeatures.entropy(df[df.columns[1]], prob='standard')
        fap = modelFeatures.falseProbAlarm(df[df.columns[0]].values, df[df.columns[1]].values)
        rcsCalculate = modelFeatures.rcs(df[df.columns[1]].values)
        stetsonKCalculate = modelFeatures.stetsonK(df[df.columns[1]].values)
        interq_range = modelFeatures.interq_range(df[df.columns[1]])
        kurtosis = modelFeatures.kurtosis(df[df.columns[1]])
        skewness = modelFeatures.skewness(df[df.columns[1]])
        calc_mean = modelFeatures.calc_mean(df[df.columns[1]])
        mean_abs_deviation = modelFeatures.mean_abs_deviation(df[df.columns[1]])
        calc_std = modelFeatures.calc_std(df[df.columns[1]])

        features = [originalPeriod, autocorr, ecdf, entropy, fap, rcsCalculate, stetsonKCalculate, interq_range, kurtosis, skewness, calc_mean, mean_abs_deviation, calc_std]

        return features
    
    if selectFeaturesDropdown_feature.value == "Only Period":
        features = []

        maxRange = 1000

        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 1000:
            maxRange = 800

        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 800:
            maxRange = 600
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 600:
            maxRange = 400
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 400:
            maxRange = 200
        
        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 200:
            maxRange = 100

        if (max(df[df.columns[0]]) - min(df[df.columns[0]])) < 100:
            maxRange = 10
        
        gatspy_period = LombScargleFast()
        gatspy_period.fit(df[df.columns[0]], df[df.columns[1]], 0.1)
        gatspy_period.optimizer.period_range=(0.1, maxRange)
        originalPeriod = gatspy_period.best_period

        fap = allFeatures.falseProbAlarm(df[df.columns[0]].values, df[df.columns[1]].values)

        features = [originalPeriod, fap]

        return features
    
    
    if selectFeaturesDropdown_feature.value == "All Features":
        
        autocorr = allFeatures.autocorr(df[df.columns[1]])
        calc_centroid = allFeatures.calc_centroid(df[df.columns[1]], fs=100)
        mean_abs_diff = allFeatures.mean_abs_diff(df[df.columns[1]])
        mean_diff = allFeatures.mean_diff(df[df.columns[1]])
        median_abs_diff = allFeatures.median_abs_diff(df[df.columns[1]])
        median_diff = allFeatures.median_diff(df[df.columns[1]])
        distance = allFeatures.distance(df[df.columns[1]])
        sum_abs_diff = allFeatures.sum_abs_diff(df[df.columns[1]])
        slope = allFeatures.slope(df[df.columns[1]])
        auc = allFeatures.auc(df[df.columns[1]], fs=100)
        pk_pk_distance = allFeatures.pk_pk_distance(df[df.columns[1]])
        entropy = allFeatures.entropy(df[df.columns[1]], prob='standard')
        neighbourhood_peaks = allFeatures.neighbourhood_peaks(df[df.columns[1]], n=10)
        hist = allFeatures.hist(df[df.columns[1]], nbins=10, r=1)
        interq_range = allFeatures.interq_range(df[df.columns[1]])
        kurtosis = allFeatures.kurtosis(df[df.columns[1]])
        skewness = allFeatures.skewness(df[df.columns[1]])
        calc_max = allFeatures.calc_max(df[df.columns[1]])
        calc_min = allFeatures.calc_min(df[df.columns[1]])
        calc_mean = allFeatures.calc_mean(df[df.columns[1]])
        calc_median = allFeatures.calc_median(df[df.columns[1]])
        mean_abs_deviation = allFeatures.mean_abs_deviation(df[df.columns[1]])
        median_abs_deviation = allFeatures.median_abs_deviation(df[df.columns[1]])
        rms = allFeatures.rms(df[df.columns[1]])
        calc_std = allFeatures.calc_std(df[df.columns[1]])
        calc_var = allFeatures.calc_var(df[df.columns[1]])
        ecdf = allFeatures.ecdf(df[df.columns[1]], d=1)[0]
        spectral_distance = allFeatures.spectral_distance(df[df.columns[1]], fs=100)
        fundamental_frequency = allFeatures.fundamental_frequency(df[df.columns[1]], fs=100)
        max_power_spectrum = allFeatures.max_power_spectrum(df[df.columns[1]], fs=100)
        max_frequency = allFeatures.max_frequency(df[df.columns[1]], fs=100)
        median_frequency = allFeatures.median_frequency(df[df.columns[1]], fs=100)
        spectral_centroid = allFeatures.spectral_centroid(df[df.columns[1]], fs=100)
        spectral_decrease = allFeatures.spectral_decrease(df[df.columns[1]], fs=100)
        spectral_kurtosis = allFeatures.spectral_kurtosis(df[df.columns[1]], fs=100)
        spectral_skewness = allFeatures.spectral_skewness(df[df.columns[1]], fs=100)
        spectral_spread = allFeatures.spectral_spread(df[df.columns[1]], fs=100)
        spectral_slope = allFeatures.spectral_slope(df[df.columns[1]], fs=100)
        spectral_variation = allFeatures.spectral_variation(df[df.columns[1]], fs=100)
        power_bandwidth = allFeatures.power_bandwidth(df[df.columns[1]], fs=100)
        fft_mean_coeff = allFeatures.fft_mean_coeff(df[df.columns[1]], fs=100, nfreq=256)
        spectral_entropy = allFeatures.spectral_entropy(df[df.columns[1]], fs=100)
        wavelet_entropy = allFeatures.wavelet_entropy(df[df.columns[1]])
        wavelet_abs_mean = allFeatures.wavelet_abs_mean(df[df.columns[1]])
        wavelet_std = allFeatures.wavelet_std(df[df.columns[1]])
        wavelet_var = allFeatures.wavelet_var(df[df.columns[1]])
        wavelet_energy = allFeatures.wavelet_energy(df[df.columns[1]])
        
        
        features = [autocorr, calc_centroid, mean_abs_diff, mean_diff, median_abs_diff, median_diff, distance, sum_abs_diff, slope, auc, pk_pk_distance, entropy, neighbourhood_peaks, hist, interq_range, kurtosis, skewness, calc_max, calc_min, calc_mean, calc_median, mean_abs_deviation, median_abs_deviation, rms, calc_std, calc_var, ecdf, spectral_distance, fundamental_frequency, max_power_spectrum, max_frequency, median_frequency, spectral_centroid, spectral_decrease, spectral_kurtosis, spectral_skewness, spectral_spread, spectral_slope, spectral_variation, power_bandwidth, fft_mean_coeff, spectral_entropy, wavelet_entropy, wavelet_abs_mean, wavelet_std, wavelet_var, wavelet_energy]

        return features

In [69]:
def cleanButtonFunction_feature(change):
    with cleanInfoOut:
        clear_output()
        print(colored("Cleaning...", 'blue', attrs=['bold']))
        
    global cleanedData_feature
    cleanedData_feature = []
    for i in os.listdir(fileDirectory_feature):
        try:
            with open(fileDirectory_feature+"//"+i, 'r') as myfile:
                with cleanInfoOut:
                    clear_output()
                    print(colored(f"Cleaning {i}", 'blue', attrs=['bold']))
                cleanedData_feature.append(dataCleaning_feature(myfile))
        
        except Exception as e:
            with open("cleaningErrorLog.txt", "a") as f:
                print(f"{i}", file=f)

    with cleanInfoOut:
        clear_output()
        print(colored("Cleaning Completed!!", 'green', attrs=['bold']))

    saveCleanedDataButton_feature.disabled = False

In [70]:
def saveCleanedDataFunction_feature(change):
    if "cleanedData" not in os.listdir("./05Feature_Detection/"):
        os.mkdir("05Feature_Detection/cleanedData")
        
    for name,data in zip(os.listdir(fileDirectory_feature), cleanedData_feature):
        data.to_csv(f"{workingDirectory_home}/05Feature_Detection/cleanedData/{name}.csv", index=None, header=None)
        
    with saveInfoOut:
        clear_output()
        print(colored("File saved!!", 'green', attrs=['bold']))

In [71]:
def extractFeaturesFunction_feature(change):
    with extractInfoOut:
        clear_output()
        print(colored("Extracting, please wait. It may take a while...", 'blue', attrs=['bold']))

    global extractedFeatures
    extractedFeatures = []
    fileName = []
    
    if dataSourceDropdown_feature.value == "ASAS":
            
        counter = 0
        for i in os.listdir(workingDirectory_home+"/05Feature_Detection/cleanedData"):
            counter += 1
            with extractInfoOut:
                clear_output()
                print(colored(f"No.: {counter}\nextracting: {i}", 'blue', attrs=['bold']))
            extractedFeatures.append(dataExtraction_feature(pd.read_csv(workingDirectory_home+"/05Feature_Detection/cleanedData/"+i, header=None)))
            fileName.append(i)
    
    if dataSourceDropdown_feature.value == "Standard":
        counter = 0
        for i in os.listdir(fileDirectory_feature):
            if i.split(".")[-1] == "csv":
                counter += 1
                with extractInfoOut:
                    clear_output()
                    print(colored(f"No.: {counter}\nextracting: {i}", 'blue', attrs=['bold']))
                extractedFeatures.append(dataExtraction_feature(pd.read_csv(fileDirectory_feature+"/"+i, header=None)))
                fileName.append(i)

    if selectFeaturesDropdown_feature.value == "Model Features":
        ef = pd.DataFrame(extractedFeatures, columns = ["period", "autocorr", "ecdf", "entropy", "fap", "rcsCalculate", "stetsonKCalculate", "interq_range", "kurtosis", "skewness", "calc_mean", "mean_abs_deviation", "calc_std"])
        ef.insert(0, "filename", fileName)
        ef.to_csv("05Feature_Detection/output/modelFeatures.csv", index=None)
        
#SR --start
        with extractMessageOut:
            clear_output()
            print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))                
#SR --end
    
        with extractInfoOut:
            clear_output()
            #print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))
            print(colored("Model Features", 'red', attrs=['bold', 'reverse']))
            display(pd.read_csv("05Feature_Detection/output/modelFeatures.csv"))
            
        
    if selectFeaturesDropdown_feature.value == "Only Period":
        ef = pd.DataFrame(extractedFeatures, columns = ["period", "fap"])
        ef.insert(0, "filename", fileName)
        ef.to_csv("05Feature_Detection/output/onlyPeriod.csv", index=None)
#SR --start
        with extractMessageOut:
            clear_output()
            print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))                
#SR --end        
        with extractInfoOut:
            clear_output()
            #print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))
            print(colored("Only Period", 'red', attrs=['bold', 'reverse']))
            display(pd.read_csv("05Feature_Detection/output/onlyPeriod.csv"))
            
    
    if selectFeaturesDropdown_feature.value == "All Features":
        ef = pd.DataFrame(extractedFeatures, columns = ["autocorr", "calc_centroid", "mean_abs_diff", "mean_diff", "median_abs_diff", "median_diff", "distance", "sum_abs_diff", "slope", "auc", "pk_pk_distance", "entropy", "neighbourhood_peaks", "hist", "interq_range", "kurtosis", "skewness", "calc_max", "calc_min", "calc_mean", "calc_median", "mean_abs_deviation", "median_abs_deviation", "rms", "calc_std", "calc_var", "ecdf", "spectral_distance", "fundamental_frequency", "max_power_spectrum", "max_frequency", "median_frequency", "spectral_centroid", "spectral_decrease", "spectral_kurtosis", "spectral_skewness", "spectral_spread", "spectral_slope", "spectral_variation", "power_bandwidth", "fft_mean_coeff", "spectral_entropy", "wavelet_entropy", "wavelet_abs_mean", "wavelet_std", "wavelet_var", "wavelet_energy"])
        ef.insert(0, "filename", fileName)
        ef.to_csv("05Feature_Detection/output/allFeatures.csv", index=None)
#SR --start
        with extractMessageOut:
            clear_output()
            print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))                
#SR --end
    
        with extractInfoOut:
            clear_output()
            #print(colored("Extraction Complete!!\n\n", 'green', attrs=['bold']))
            print(colored("All Features", 'red', attrs=['bold', 'reverse']))
            display(pd.read_csv("05Feature_Detection/output/allFeatures.csv"))
        

In [72]:
# workingDirectoryButton_feature = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_feature.title = '<b>Working Directory</b>'
# workingDirectoryButton_feature.show_only_dirs = True

fileDirectoryButton_feature = ipyfilechooser.FileChooser(workingDirectory_home)
fileDirectoryButton_feature.title = '<b>Select Light Curve Directory</b>'

dataSourceDropdown_feature = widgets.Dropdown(description="<b>Data Source</b>", options=["Standard", "ASAS"])

uploadProceedButton_feature = widgets.Button(description="Load Data", button_style='warning', disabled=True)
uploadInfoOut = widgets.Output()

cleanButton_feature = widgets.Button(description="Clean Data", disabled=True, icon='broom', button_style='primary')
cleanInfoOut = widgets.Output()

saveCleanedDataButton_feature = widgets.Button(description="Save Data", disabled=True, icon='save', button_style='success')
saveInfoOut = widgets.Output()
selectFeaturesDropdown_feature = widgets.RadioButtons(description="<b>Features</b>",options=["Model Features", "Only Period","Fractal Features", "All Features"], disabled=True, value="Model Features")

extractFeaturesButton_feature = widgets.Button(description="Extract", disabled=True, icon='download', button_style='info')
extractMessageOut = widgets.Output()
extractInfoOut = widgets.Output()

In [73]:
uploadProceedButton_feature.on_click(uploadProceedFunction_feature)
cleanButton_feature.on_click(cleanButtonFunction_feature)
extractFeaturesButton_feature.on_click(extractFeaturesFunction_feature)
saveCleanedDataButton_feature.on_click(saveCleanedDataFunction_feature)

In [74]:
#featureExtTab = widgets.VBox([fileDirectoryButton_feature, dataSourceDropdown_feature, uploadProceedButton_feature, uploadInfoOut, widgets.HBox([widgets.VBox([cleanButton_feature, cleanInfoOut]), widgets.VBox([saveCleanedDataButton_feature, saveInfoOut])]), selectFeaturesDropdown_feature, extractFeaturesButton_feature, extractInfoOut])

#SR --start
top = widgets.Box(children=[fileDirectoryButton_feature], layout=Layout(display='flex'))
left = widgets.VBox([dataSourceDropdown_feature, uploadProceedButton_feature, uploadInfoOut, widgets.HBox([widgets.VBox([cleanButton_feature, cleanInfoOut]), widgets.VBox([saveCleanedDataButton_feature, saveInfoOut])]), selectFeaturesDropdown_feature, extractFeaturesButton_feature, extractMessageOut])
right = widgets.Box(children=[extractInfoOut], layout=Layout(display='flex', height='90vh'))
body = widgets.GridBox(children = [left,right], layout=Layout(grid_template_columns='25% 75%'))
featureExtTab = widgets.GridBox(children = [top,body])
#SR --end

In [75]:
# Classify Variables Tab

from ipywidgets import widgets
from IPython.display import display, clear_output
import ipyfilechooser
import pickle
import os
import pandas as pd
import joblib
from model.assets import asiva_model, asiva_cm

In [76]:
def proceedFunction_prediction(change):
#     global workingDirectory_prediction
    global df_prediction
    global x_test_prediction
    global clf
    global image_prediction
    global img_prediction
    
#     workingDirectory_prediction = workingDirectoryButton_prediction.selected_path
#     os.chdir(workingDirectory_prediction)

    if "06Classify_Variables" not in os.listdir(workingDirectory_home):
        os.mkdir("06Classify_Variables")
    
    if "output" not in os.listdir("./06Classify_Variables/"):
        os.mkdir("06Classify_Variables/output")
    
    df_prediction = pd.read_csv(fileNameButton_prediction.selected_path + "/" + fileNameButton_prediction.selected_filename)
    
    with dfOut_prediction:
        clear_output()
        display(df_prediction)
    
    x_test_prediction = df_prediction.iloc[:, 1:].values
    
    if modelSelectionDropdown_prediction.value == "ASIVA":
    
        clf = asiva_model()
        image_prediction = asiva_cm()
        img_prediction = widgets.Image(value=image_prediction, format='png', width=600, height=600)
        
    if modelSelectionDropdown_prediction.value == "Custom":
        
        try:
            clf = joblib.load(customModelButton_prediction.selected_path+"/"+customModelButton_prediction.selected_filename)
        except:
            clf = pickle.load(customModelButton_prediction.selected_path+"/"+customModelButton_prediction.selected_filename)
        

    with confusionMatrixOut_prediction:
        clear_output()
        
        if modelSelectionDropdown_prediction.value == "ASIVA":
            clear_output()
            print(colored("Confusion Matrix", 'red', attrs=['bold', 'reverse']))
            display(img_prediction)
        
        if modelSelectionDropdown_prediction.value == "Custom":
            clear_output()
            print(colored("Custom Model", 'red', attrs=['bold', 'reverse']))
    
    with proceedOut_prediction:
        clear_output()
        print(colored("Model loaded!!", 'green', attrs=['bold']))

    predictButton_prediction.disabled = False

In [77]:
def predictFunction_prediction(change):
    global y_pred_prediction
    
    y_pred_prediction = clf.predict(x_test_prediction)
    
    y_prob_prediction = clf.predict_proba(x_test_prediction)
    
    firstClass = []
    secondClass = []
    firstProb = []
    secondProb = []
    
    for i in range(y_prob_prediction.shape[0]):
        one = np.argsort(y_prob_prediction[i])[-1]
        two = np.argsort(y_prob_prediction[i])[-2]

        firstClass.append(one)
        firstProb.append(y_prob_prediction[i][one])
        if y_prob_prediction[i][two] != 0:
            secondClass.append(two)
            secondProb.append(y_prob_prediction[i][two])
        else:
            secondClass.append(np.nan)
            secondProb.append(np.nan)
            
    if modelSelectionDropdown_prediction.value == "ASIVA" or modelSelectionDropdown_prediction.value == "Custom":
        
        replacements = {
                        0: "DCEP",
                        1: "DSCT",
                        2: "EC",
                        3: "ED",
                        4: "ESD",
                        5: "MIRA",
                        6: "RRAB",
                        7: "RRC"
                        }

        y_pred_prediction = [replacements.get(x, x) for x in y_pred_prediction]

        firstClass = [replacements.get(x, x) for x in firstClass]
        secondClass = [replacements.get(x, x) for x in secondClass]
    
    predicteddf_prediction = pd.DataFrame({"Light Curve": df_prediction["filename"], "Class 01": firstClass, "Prob 01 (%)": firstProb, "Class 02": secondClass, "Prob 02 (%)": secondProb})
    predicteddf_prediction.to_csv("06Classify_Variables/output/predictedResults.csv", index=None)
    
    with predictOut_prediction:
        clear_output()
        print(colored("Prediction Results", 'red', attrs=['bold', 'underline'])) 
        display(predicteddf_prediction)

In [78]:
# workingDirectoryButton_prediction = ipyfilechooser.FileChooser(os.path.expanduser("~"))
# workingDirectoryButton_prediction.title = '<b>Working Directory</b>'
# workingDirectoryButton_prediction.show_only_dirs = True

fileNameButton_prediction = ipyfilechooser.FileChooser(workingDirectory_home)
fileNameButton_prediction.title = '<b>Load Features</b>'
fileNameButton_prediction.filter_pattern = '*.csv'

modelSelectionDropdown_prediction = widgets.Dropdown(description="<b>Model</b>", options=["ASIVA", "Custom"], value="ASIVA")

customModelButton_prediction = ipyfilechooser.FileChooser(workingDirectory_home)
customModelButton_prediction.title = '<b>Load Custom Model</b>'
customModelButton_prediction.filter_pattern = ['*.joblib', '*.pickle', '*.pkl', '*.h5']

accordian_model = widgets.Accordion(children=[customModelButton_prediction])

accordian_model.selected_index = None
accordian_model.set_title(index=0, title="Custom Model")

proceedButton_prediction = widgets.Button(description="Load Model", button_style='warning', disabled=True)
dfOut_prediction = widgets.Output()
proceedOut_prediction = widgets.Output()
confusionMatrixOut_prediction = widgets.Output()

predictButton_prediction = widgets.Button(description="Predict", disabled=True, icon='bolt', button_style='info')
predictOut_prediction = widgets.Output()

In [79]:
proceedButton_prediction.on_click(proceedFunction_prediction)
predictButton_prediction.on_click(predictFunction_prediction)

In [80]:
#classifyVarTab = widgets.VBox([fileNameButton_prediction, widgets.HBox([modelSelectionDropdown_prediction, accordian_model]), proceedButton_prediction, proceedOut_prediction, predictButton_prediction, widgets.HBox([confusionMatrixOut_prediction, predictOut_prediction])])

#SR --start
top = widgets.Box(children=[fileNameButton_prediction], layout=Layout(display='flex'))
left_up = widgets.VBox([ widgets.HBox([modelSelectionDropdown_prediction, accordian_model]), proceedButton_prediction, proceedOut_prediction, predictButton_prediction])
left = widgets.GridBox(children=[left_up, confusionMatrixOut_prediction], layout=Layout(grid_gap='10px 0px'))
right = widgets.Box(children=[predictOut_prediction], layout=Layout(display='flex', height='80vh'))
body = widgets.GridBox(children=[left,right], layout=Layout(grid_template_columns='40% 60%'))
classifyVarTab = widgets.GridBox(children=[top,body])
#SR --end

In [81]:
Title = "ASIVA v1.5.2"
aboutUsTitle = widgets.HTML(value = f"<b><font color='red'> <font size=5>{Title}</b>")
aboutUsDescription = widgets.Label("Description: AstroStatistics In Variable Astronomy")
aboutUsAuthors = widgets.Label("Authors: P. Saleh, T. Singh & D. Hazarika")

aboutUsTab = widgets.VBox([aboutUsTitle, aboutUsDescription, aboutUsAuthors])

In [82]:
# Combined Tab

tab = widgets.Tab()
tab.children = [homeTab, dataprocTab, vardetectTab, stationaryTab, perioddetectTab, featureExtTab, classifyVarTab, aboutUsTab]
tab.set_title(index=0,title="Home")
tab.set_title(index=1,title="Data Processing") 
tab.set_title(index=2,title="Variable Detection")
tab.set_title(index=3,title="Stationarity Test")
tab.set_title(index=4,title="Period Detection")
tab.set_title(index=5,title="Feature Extraction")
tab.set_title(index=6,title="Classify Variables")
tab.set_title(index=7,title="About Us")

In [83]:
tab